In [1]:
import numpy as np
import sonnet as snt
import tensorflow as tf
from sklearn import tree
import copy
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

H:\Miniconda\envs\sounds-deep\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
H:\Miniconda\envs\sounds-deep\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
H:\Miniconda\envs\sounds-deep\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
H:\Miniconda\envs\sounds-deep\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
H:\Miniconda\envs\sounds-deep\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary inc

In [4]:
def load_celeba(data_dir, restricted_degree, label_type, print_ratio=False):
    """Returns CelebA as (train_data, train_labels, test_data, test_labels)

        Shapes are (162770, 64, 64, 3), (162770, 2), (19962, 64, 64, 3), (19962, 10)
        Data is in [0,1] and labels are one-hot

        Arg:
          restricted_degree: only keep the instances with at least d selected attributes
    """
    train_data = np.load(os.path.join(data_dir, 'celeba_train_imgs.npy'))
    test_data = np.load(os.path.join(data_dir, 'celeba_test_imgs.npy'))

    info_pak = np.load(os.path.join(data_dir, 'celeba_attr.npz'))
    train_idxs = info_pak['train_idxs']
    val_idxs = info_pak['val_idxs']
    test_idxs = info_pak['test_idxs']
    attribute_names = info_pak['attribute_names']
    attributes = info_pak['attributes']
    male_attr_idx = 20

    def get_label(data, idxs):
        label = attributes[idxs]

        if label_type == 'gender':
            label = 1-label[:, male_attr_idx].reshape([-1, 1])
            label = np.append(label, 1 - label, 1)
        elif label_type == 'subattr':
            pass
        return data, label

    train_data, train_label = get_label(train_data, train_idxs)
    test_data, test_label = get_label(test_data, test_idxs)

    if print_ratio:
        print('\nCelebA restricted degree: {}'.format(restricted_degree))
        train_ratio = sum(train_label[:, 1]) / train_label.shape[0]
        test_ratio = sum(test_label[:, 1]) / test_label.shape[0]
        print('Training set - Male: {:.2f}% ({}/{}), Not male: {:.2f}%'.format(train_ratio * 100,
                                                                               sum(train_label[:, 1]),
                                                                               train_label.shape[0],
                                                                               100 - train_ratio * 100))
        print('Testing set - Male: {:.2f}% ({}/{}), Not male: {:.2f}%'.format(test_ratio * 100,
                                                                              sum(test_label[:, 1]),
                                                                              test_label.shape[0],
                                                                              100 - test_ratio * 100))

    return train_data, train_label, test_data, test_label

In [5]:
train_data, train_labels, test_data, test_labels = load_celeba(
        'H:\\CodeRange\\CelebA\\npy\\', restricted_degree=0, print_ratio=False, label_type='subattr')

In [6]:
attributes_names = np.load('H:\\CodeRange\\CelebA\\npy\\celeba_attr.npz')['attribute_names']

In [7]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
model=tf.saved_model.loader.load(session, [tf.saved_model.tag_constants.SERVING], './saved_model/SkewRank06_High_Cheekbones/saved_model')
with open('./saved_model/SkewRank06_High_Cheekbones/saved_model/decision_tree.pkl', 'rb') as dt_file:
    decision_tree = pickle.load(dt_file)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./saved_model/SkewRank06_High_Cheekbones/saved_model\variables\variables


H:\Miniconda\envs\sounds-deep\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
def get_variables(model_meta):
    graph = tf.get_default_graph()
    sig_vars = copy.deepcopy(model_meta.signature_def['serving_default'])
    sig_inputs = sig_vars.inputs
    sig_outputs = sig_vars.outputs
    output = dict()
    for k in sig_inputs.keys():
        print('{:20}, {}'.format(k,sig_inputs[k].name))
        output[k] = graph.get_tensor_by_name(sig_inputs[k].name)
    for k in sig_outputs.keys():
        print('{:20}, {}'.format(k,sig_outputs[k].name))
        output[k] = graph.get_tensor_by_name(sig_outputs[k].name)
    return output

tensors = get_variables(model)

t_x = tensors['x']
t_latent_var = tensors['latent_var']
t_output = tensors['output']

x                   , data_ph:0
latent_var          , vae_1/MultivariateNormalDiag/sample/affine_linear_operator/forward/add:0
output              , vae_1/batch_apply/Reshape_1:0


In [9]:
epoch_size = 32

def get_latent_var_list(data):
    test_data = data
    test_data_len = test_data.shape[0]
    epoch_num = test_data_len // epoch_size
    # epoch_num = 60
    instance_num = epoch_num * epoch_size

    latent_var = []
    for i in tqdm(range(epoch_num)):
        epoch_beg = i*epoch_size
        epoch_end = (i+1)*epoch_size
        epoch_input = test_data[epoch_beg:epoch_end].astype('float32') / 255.0
    #     print(epoch_beg, epoch_end)
        outputs = session.run([t_output, t_latent_var], 
                              feed_dict={t_x:epoch_input})
        latent_var.append(outputs[1])
    return latent_var

train_latent_var_lists = get_latent_var_list(train_data)
test_latent_var_lists = get_latent_var_list(test_data)

100%|████████████████████████████████████████████████████████████| 623/623 [00:10<00:00, 57.99it/s]


In [10]:
train_latent_var_lists = np.array(train_latent_var_lists).reshape([-1, 50])
test_latent_var_lists = np.array(test_latent_var_lists).reshape([-1, 50])

In [16]:
te_label = 1-test_labels[:19936, 19]
tr_label = 1-train_labels[:162752, 19]
decision_tree.score(test_latent_var_lists, te_label)

0.8281500802568218

In [ ]:
train_latent_var_lists.shape

In [ ]:
attributes_names = np.load('./gender-classification/data/toy_celeba/celeba_attr.npz')['attribute_names']
for i,ac in enumerate(accu):
    print('{:4}: {:20}, {:.4f}'.format(i, attributes_names[i], ac))

In [ ]:
# Example

idx = 20

outputs = session.run([t_output, t_latent_var], feed_dict={t_x:train_data[:32]})

plt.imshow(train_data[idx])
plt.show()

z = outputs[1]
y = outputs[0]
temp = y[0, idx, :]
plt.imshow(temp)
plt.show()

print('Male:', train_latent_labels[idx][20])
print('Mustache:', train_latent_labels[idx][22])
print('No_Beard:', train_latent_labels[idx][24])

In [ ]:
path=decision_tree.decision_path([z[0][idx]])
for i,x in enumerate(z[0][idx]):
    print(i, x)

In [ ]:
print(path)

In [ ]:
tree.export_graphviz(
        decision_tree,
        out_file='./tree.dot',
        class_names=['negative', 'positive'],
        filled=True,
        rounded=True,
        proportion=True)

## Graphviz

In [ ]:
def add_info_to_node_label(in_filename, out_filename, node_id, info):
    with open(in_filename) as f:
        linelist = f.readlines()
    for i,s in enumerate(linelist):
        if s.startswith('{} [label='.format(node_id)):
            idx = s.find('", fillcolor=')
            s = s[:idx] + "\\n" + info + s[idx:]
            linelist[i] = s
            break
    with open(out_filename, "w+") as f:
        for x in linelist:
            f.write(x)


## Other

In [ ]:
for i,x in enumerate(attributes_names):
    print(i, x)